# Image rotation with CNN

this is a search and development project to do on the weekends.


my main resourse is this blog post: https://d4nst.github.io/2017/01/12/image-orientation/



In [51]:
targetUrl = 'datasets/streetview'
gmap_images_url = "http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/images"


## Rotate images
#### Get images

In [52]:
import math
# generate filenames
# example 000005_3.jpg
# the postfix can be a number between 0 and 5
# the first part is a simple xml padded to 6 char: '{0:0>6}'.format('132')
fileNames = []
max = 10
for i in range(1, math.floor(max/6)+1):
    baseName = '{0:0>6}'.format(i);
    for postfix in range(6):
        fileNames.append(baseName + "_" + str(postfix) + ".jpg");


In [53]:
fileNames[0:10]

['000001_0.jpg',
 '000001_1.jpg',
 '000001_2.jpg',
 '000001_3.jpg',
 '000001_4.jpg',
 '000001_5.jpg',
 '000002_0.jpg',
 '000002_1.jpg',
 '000002_2.jpg',
 '000002_3.jpg']

In [54]:
import urllib.request


for filename in fileNames:
    print('downloading {0}\n'.format(filename))
    urllib.request.urlretrieve(gmap_images_url + '/' + filename, targetUrl + '/' + filename)


downloading 000001_0.jpg

downloading 000001_1.jpg

downloading 000001_2.jpg

downloading 000001_3.jpg

downloading 000001_4.jpg

downloading 000001_5.jpg

downloading 000002_0.jpg

downloading 000002_1.jpg

downloading 000002_2.jpg

downloading 000002_3.jpg

downloading 000002_4.jpg

downloading 000002_5.jpg



### rotate image in 3 orientation, copy and add some label

labels: https://www.impulseadventure.com/photo/exif-orientation.html

- 1 top, 
- 8 left side, 
- 3 bottom, 
- 6 right side

prepend this label to the filename separted by a "#", eg. `1#000001_5.jpg`



In [55]:
import os
from PIL import Image

ORIENTATION_LABELS = { 'top': 1, 'left':  8, 'bottom': 3, 'right': 6 }


def rotateImage(orientation, fileName, labeledFileName, subdir):
    # crop the image to have equal sizes
    img = Image.open(os.path.join(subdir, fileName))
    size = 0, 0, 1024, 1024
    image = img.crop(size)
    
    targetPath = os.path.join(subdir, labeledFileName)
    
    # don't rotate the original
    if orientation == ORIENTATION_LABELS['top']:
        image.save(targetPath)
        return
    
    anglesForOrientation = { ORIENTATION_LABELS['top']: 0, 
                             ORIENTATION_LABELS['left']: 90, 
                             ORIENTATION_LABELS['bottom']: 180, 
                            ORIENTATION_LABELS['right']: 270 }
    
    if not orientation in anglesForOrientation.keys():
        raise Exception('Unknown orienation: ' + orientation)
    
    angle = anglesForOrientation[orientation]
   
    rotated = image.rotate(angle)
    rotated.save(targetPath)
                              
#rotateImage(ORIENTATION_LABELS['bottom'], 1, 2, 3)

In [56]:
# first iterate them and add the appropriate label
import os


def isImage(file):
    return '.jpg' in file

def isProcessed(file):
    return '#' in file

def addLabelToFile(label, file):
    return '{label}#{file}'.format(label = label, file = file)


def rotateImages(fileName, subdir):
    for orientation in ORIENTATION_LABELS.values():
        labeledFileName = addLabelToFile(orientation, fileName)
        rotatedImage = rotateImage(orientation, fileName, labeledFileName, subdir)




for subdir, dirs, files in os.walk(targetUrl):
    for file in files:
        if (not isImage(file)):
            continue
        if (isProcessed(file)):
            continue
        print ('Processing: {0}'.format(os.path.join(subdir, file)))
        
        
        # create the rotations
        rotateImages(file, subdir)
        
        # create the default top orientation - it must be here to have the original file
        labeled = addLabelToFile(ORIENTATION_LABELS['top'], file)
        origPath = os.path.join(subdir, file)
        labeledPath = os.path.join(subdir, labeled)
        os.rename(origPath, labeledPath)
        
        

Processing: datasets/streetview/000001_5.jpg
Processing: datasets/streetview/000001_4.jpg
Processing: datasets/streetview/000001_3.jpg
Processing: datasets/streetview/000001_2.jpg
Processing: datasets/streetview/000001_0.jpg
Processing: datasets/streetview/000001_1.jpg
Processing: datasets/streetview/000002_4.jpg
Processing: datasets/streetview/000002_5.jpg
Processing: datasets/streetview/000002_2.jpg
Processing: datasets/streetview/000002_3.jpg
Processing: datasets/streetview/000002_1.jpg
Processing: datasets/streetview/000002_0.jpg


balint#woah.jpg
